In [2]:
import tensorflow as tf

def get_weights(n_features, n_labels):
    """
    Return TensorFlow weights
    :param n_features: Number of features
    :param n_labels: Number of labels
    :return: TensorFlow weights
    """
    # Return weights
    weights = tf.Variable(tf.truncated_normal((n_features, n_labels)))
    return weights


def get_biases(n_labels):
    """
    Return TensorFlow bias
    :param n_labels: Number of labels
    :return: TensorFlow bias
    """
    # Return biases
    # Since the weights are already helping prevent the model from getting stuck, 
    # I do not randomize the bias. Let's use the simplest solution, setting the bias to 0.
    bias = tf.Variable(tf.zeros(n_labels))
    return bias


def linear(input, w, b):
    """
    Return linear function in TensorFlow
    :param input: TensorFlow input
    :param w: TensorFlow weights
    :param b: TensorFlow biases
    :return: TensorFlow linear function
    """
    # Linear Function (xW + b)
    return tf.add(tf.matmul(input, w), b)

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

def mnist_features_labels(n_labels):
    """
    Gets the first <n> labels from the MNIST dataset
    :param n_labels: Number of labels to use
    :return: Tuple of feature list and label list
    """
    mnist_features = []
    mnist_labels = []

    mnist = input_data.read_data_sets('/datasets/ud730/mnist', one_hot=True)

    # In order to make quizzes run faster, we're only looking at 10000 images
    for mnist_feature, mnist_label in zip(*mnist.train.next_batch(10000)):

        # Add features and labels if it's for the first <n>th labels
        if mnist_label[:n_labels].any():
            mnist_features.append(mnist_feature)
            mnist_labels.append(mnist_label[:n_labels])

    return mnist_features, mnist_labels


# Number of features (28*28 image is 784 features)
n_features = 784
# Number of labels
n_labels = 3

# Features and Labels
features = tf.placeholder(tf.float32)
labels = tf.placeholder(tf.float32)

# Weights and Biases
w = get_weights(n_features, n_labels)
b = get_biases(n_labels)

# Linear Function xW + b
logits = linear(features, w, b)

# Training data
train_features, train_labels = mnist_features_labels(n_labels)

with tf.Session() as session:
    # Initialize session variables
    session.run(tf.global_variables_initializer())
    
    # Softmax
    prediction = tf.nn.softmax(logits)

    # Cross entropy
    # This quantifies how far off the predictions were.
    # You'll learn more about this in future lessons.
    cross_entropy = -tf.reduce_sum(labels * tf.log(prediction), reduction_indices=1)

    # Training loss
    # You'll learn more about this in future lessons.
    loss = tf.reduce_mean(cross_entropy)

    # Rate at which the weights are changed
    # You'll learn more about this in future lessons.
    learning_rate = 0.08

    # Gradient Descent
    # This is the method used to train the model
    # You'll learn more about this in future lessons.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Run optimizer and get loss
    _, l = session.run(
        [optimizer, loss],
        feed_dict={features: train_features, labels: train_labels})

# Print loss
print('Loss: {}'.format(l))


Extracting /datasets/ud730/mnist/train-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist/train-labels-idx1-ubyte.gz
Extracting /datasets/ud730/mnist/t10k-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist/t10k-labels-idx1-ubyte.gz
Loss: 5.565894603729248


In [4]:
### GRADER
### Be sure to run all cells above before running this cell ###

import numpy as np
import tensorflow as tf
from tensorflow.python.framework.errors import FailedPreconditionError
import os
import sys

def is_weights_good(w):
    w_answer = [[-0.01811021, 0.51838219],
 [ 0.05832404, -0.48847285],
 [-0.37598562, -0.77113968],
 [-0.59224647, -0.31185189],
 [ 0.21055079, -1.1010232 ]] 
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        w_result = sess.run(w)

    return np.allclose(w_answer, w_result)


def is_biases_good(b):
    b_answer = [0.0, 0.0]
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        b_result = sess.run(b)
        
    return np.array_equal(b_answer, b_result)


def is_linear_good(l, test_input):
    
    logits_answer = [[-2.34565091, -9.52450562], [-8.03849602, -9.28480148]]

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        logits_result = sess.run(l, feed_dict={test_input: [[1,2,3,4,5], [6,7,8,9,0]]})

    return np.allclose(logits_answer, logits_result)

def get_result(get_weights, get_biases, linear):
    result = {
        'correct': False,
        'feedback': 'That\'s the wrong answer.',
        'comment': ''}

    tf.set_random_seed(123456)
    
    n_features = 5
    n_labels = 2
    test_input = tf.placeholder(tf.float32)
    
    weights = get_weights(n_features, n_labels)
    biases = get_biases(n_labels)
    lin = linear(test_input, weights, biases)

    if not isinstance(weights, tf.Variable):
        result['feedback'] = 'Function weights not returning tf.Variable type.'
        result['comment'] = 'Use the tf.Variable function.'
    elif not isinstance(biases, tf.Variable):
        result['feedback'] = 'Function biases not returning tf.Variable type.'
        result['comment'] = 'Use the tf.Variable function.'
    elif weights.get_shape() != (n_features, n_labels):
        result['feedback'] = 'Function weights is returning the wrong shape.'
    elif biases.get_shape() != n_labels:
        result['feedback'] = 'Function biases is returning the wrong shape.'
    elif not is_weights_good(weights):
        result['feedback'] = 'Function weights isn\'t correct.'
    elif not is_biases_good(biases):
        result['feedback'] = 'Function biases isn\'t correct.'
    elif not is_linear_good(lin, test_input):
        import pdb;pdb.set_trace()
        result['feedback'] = 'Function linear isn\'t correct.'
    else:
        try:
            std_out = sys.stdout
            f = open(os.devnull, 'w')
            sys.stdout = f
        
        except FailedPreconditionError as err:
            if err.message.startswith('Attempting to use uninitialized value Variable'):
                result['feedback'] = 'At least one variable is not initialized.'
            else:
                raise err
        else:
            result['correct'] = True
            result['feedback'] = 'You got it!  That\'s the correct answer.'
        finally:
            sys.stdout = std_out
    return result

def run_grader(get_weights, get_biases, linear):
    
    try:
    # Get grade result information
        result = get_result(get_weights, get_biases, linear)
    except Exception as err:
        # Default error result
        result = {
            'correct': False,
            'feedback': 'Something went wrong with your submission:',
            'comment': str(err)}

    feedback = result.get('feedback')
    comment = result.get('comment')

    print("{}\n{}\n".format(feedback, comment))



if __name__ == "__main__":
    try:
        run_grader(get_weights, get_biases, linear)
    except Exception as err:
        print(str(err))

You got it!  That's the correct answer.


